In [1]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df = df.drop(columns=['Id'])

In [ ]:
# df_num = df.select_dtypes(include = ['float64', 'int64'])
# df = df_num[['LotArea','MasVnrArea','TotalBsmtSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','SalePrice']]
# df = df.dropna()
# df.head()

In [2]:
# split data set into train and validation
train = df.sample(frac=0.8, random_state=200)
validation = df.drop(train.index)
print(train.shape)
print(validation.shape)

(1168, 80)
(292, 80)


In [3]:
# split the columns into features and target; ie x y split

target = 'SalePrice'
train_x = train.drop(target, axis=1)
train_y = train[target]

validation_x = validation.drop(target, axis=1)
validation_y = validation[target]

train_x.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
863,20,RL,70.0,7931,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,NaN,0,7,2009,WD,Normal
874,50,RM,52.0,5720,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2009,WD,Abnorml
1448,50,RL,70.0,11767,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdWo,NaN,0,5,2007,WD,Normal
1425,20,RL,80.0,10721,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,10,2008,WD,Normal
991,70,RM,121.0,17671,Pave,Grvl,Reg,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# create the model
rfr_model = RandomForestRegressor(n_estimators=100, max_depth=10)

# fit the model
rfr_model.fit(train_x, train_y)

# make predictions
validation_predictions = rfr_model.predict(validation_x)

# print the mean squared error
print(mean_squared_error(validation_y, validation_predictions))

In [ ]:
from sklearn.metrics import r2_score

print(r2_score(validation_y, validation_predictions))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# create the model
gbr_model = GradientBoostingRegressor(loss='squared_error')

gbr_model.fit(train_x, train_y)

print(gbr_model.score(validation_x, validation_y))

In [ ]:
import tensorflow_decision_forests as tfdf

# Convert the dataset into a TensorFlow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label='SalePrice', task=tfdf.keras.Task.REGRESSION)
validation_ds = tfdf.keras.pd_dataframe_to_tf_dataset(validation, label='SalePrice', task=tfdf.keras.Task.REGRESSION)

print(validation_ds.dtype)

# # Train the model
# df_model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
# df_model.fit(train_ds)
# 
# # Evaluate the model
# evaluation = df_model.evaluate(validation_ds)
# print(evaluation)